#Language Learning Assistant (English - French)


##Training

###Setup
Library Installation

In [ ]:
!pip install transformers datasets peft

In [ ]:
!pip install --upgrade transformers peft

Load pretrained T5 model and T5 tokenizer

T5-small model has 60M parameters.

T5-Base: 220M
T5-Large: 770M
T5-3B: 3 billion (3B)
T5-11B: 11 billion (11B)

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

# Load pre-trained T5 model and tokenizer.
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Sample data for training and validation.

In [3]:

data = {
    "train": [
        {"input_text": "translate English to French: How are you?", "target_text": "Comment ça va?"},
        {"input_text": "translate English to French: I love programming.", "target_text": "J'aime programmer."},
        {"input_text": "translate English to French: This is a beautiful day.", "target_text": "C'est une belle journée."}
    ],
    "validation": [
        {"input_text": "translate English to French: What is your name?", "target_text": "Comment vous appelez-vous?"},
        {"input_text": "translate English to French: I am learning French.", "target_text": "J'apprends le français."}
    ]
}

# Convert to Hugging Face Datasets.
train_dataset = Dataset.from_dict({
    "input_text": [ex["input_text"] for ex in data["train"]],
    "target_text": [ex["target_text"] for ex in data["train"]]
})
val_dataset = Dataset.from_dict({
    "input_text": [ex["input_text"] for ex in data["validation"]],
    "target_text": [ex["target_text"] for ex in data["validation"]]
})


###Implement Low Rank Adaptation
Try out with r=8, Sequence to sequence language model

In [4]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Wrap the model with LoRA.
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Prints the number of trainable parameters.


trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


###Training parameters

Use FP16 for half precision floating point with 16-bit floating point number for faster computation with GPU and enabling mixed precision training

Memory needed: Each parameter in FP16 takes 2 bytes (16 bits)

Total memory needed = Number of parameters × Size per parameter

60
𝑀
×
2
 bytes
=
120
 MB
60M×2 bytes=120 MB

In [5]:
training_args = TrainingArguments(
    output_dir="./t5-lora-language-learning",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision if your GPU supports it.
    push_to_hub=False,
    remove_unused_columns=False,
)




/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


###Preprocess the tokens
Preprocess so that the input and target tokens always have same length


In [6]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


Remove input text and target text columns for training

In [7]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["input_text", "target_text"]
)
val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["input_text", "target_text"]
)

# Ensure only the required columns are returned.
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)


Training setup

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,  # This will pad your inputs to a uniform length.
)


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sabidbinhabib (sabidbinhabib-indiana-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,18.144630
2,No log,17.957535
3,No log,17.647041


TrainOutput(global_step=6, training_loss=17.041831970214844, metrics={'train_runtime': 37.8101, 'train_samples_per_second': 0.238, 'train_steps_per_second': 0.159, 'total_flos': 306557485056.0, 'train_loss': 17.041831970214844, 'epoch': 3.0})

###Save the trained model and the tokenizer

In [11]:
model.save_pretrained("./t5-lora-language-learning")
tokenizer.save_pretrained("./t5-lora-language-learning")


('./t5-lora-language-learning/tokenizer_config.json',
 './t5-lora-language-learning/special_tokens_map.json',
 './t5-lora-language-learning/spiece.model',
 './t5-lora-language-learning/added_tokens.json')

##Setup for CUDA

In [12]:
import torch

# Determine device and move the model to the device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare the input and move it to the same device.
input_text = "translate English to French: I am learning to code."
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output on the same device.
outputs = model.generate(**inputs, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Je apprends à coder.


##Model deployment

###Install local tunnel

In [13]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

###Local Tunnel setup

In [14]:
!npm config set prefix ~/.npm-global
!export PATH=~/.npm-global/bin:$PATH
!npm install -g localtunnel


⠙⠹⠸⠼
added 22 packages in 735ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

###Translator

In [ ]:
import threading
from flask import Flask, request, render_template_string
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize Flask app.
app = Flask(__name__)

# Set the directory where your fine-tuned model is stored.
model_dir = "./t5-lora-language-learning"  # Adjust this path if needed.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model.
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.to(device)

# Define a simple HTML template for the translator interface.
html_template = """
<!doctype html>
<html>
<head>
  <title>English to French Translator</title>
</head>
<body>
  <h1>English to French Translator</h1>
  <form method="POST">
    <textarea name="input_text" rows="4" cols="50" placeholder="Enter text in English..."></textarea><br>
    <input type="submit" value="Translate">
  </form>
  {% if output_text %}
    <h2>Translated Text:</h2>
    <p>{{ output_text }}</p>
  {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def translate():
    output_text = None
    if request.method == 'POST':
        input_text = request.form['input_text']
        # Prepend the task-specific prefix required by T5.
        prompt = f"translate English to French: {input_text}"
        # Tokenize the input and move tensors to the appropriate device.
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        # Generate the translation.
        outputs = model.generate(**inputs, max_length=50)
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return render_template_string(html_template, output_text=output_text)

def run_app():
    # Run on all available IP addresses, port 5000.
    app.run(host="0.0.0.0", port=5000)

# Start the Flask app in a background thread.
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


###Gamification

In [15]:
import threading
from flask import Flask, session, request, render_template_string, url_for
import random
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

app = Flask(__name__)
app.secret_key = "some"  # Replace with a secure key

# Load your fine-tuned model and tokenizer.
model_dir = "./t5-lora-language-learning"  # Adjust the path as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.to(device)

# List of English sentences for the game.
english_sentences = [
    "Good morning",
    "How are you?",
    "I love you",
    "Thank you",
    "Good night"
]

# A bank of common French phrases to serve as distractors.
distractor_bank = [
    "Bonsoir",
    "Merci beaucoup",
    "Bonne nuit",
    "Au revoir",
    "Comment ça va?",
    "Je ne sais pas",
    "S'il vous plaît",
    "Pardon"
]

def generate_translation(english_sentence):
    """Generate French translation using the fine-tuned model."""
    prompt = f"translate English to French: {english_sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=50)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

@app.route("/", methods=["GET", "POST"])
def index():
    # Initialize session variables if not already set.
    if "score" not in session or "current_question" not in session:
        session["score"] = 0
        session["current_question"] = 0

    current = session["current_question"]

    # If all questions have been answered, display final score.
    if current >= len(english_sentences):
        final_score = session["score"]
        total = len(english_sentences)
        session.clear()  # Clear session to allow a new game.
        return render_template_string("""
            <h1>Game Over</h1>
            <p>Your final score is: {{ score }} out of {{ total }}</p>
            <a href="{{ url_for('index') }}">Play Again</a>
        """, score=final_score, total=total)

    # Process answer submission.
    if request.method == "POST":
        selected = request.form.get("option")
        correct_answer = session.get("correct_answer", "")
        if selected == correct_answer:
            session["score"] += 1
            feedback = "Correct!"
        else:
            feedback = f"Incorrect. The correct answer was: {correct_answer}"
        session["current_question"] = current + 1
        return render_template_string("""
            <h1>{{ feedback }}</h1>
            <p>Current Score: {{ score }}</p>
            <a href="{{ url_for('index') }}">Next Question</a>
        """, feedback=feedback, score=session["score"])

    # For GET requests, generate a new question.
    english_sentence = english_sentences[current]
    correct_translation = generate_translation(english_sentence)
    session["correct_answer"] = correct_translation

    # Prepare distractors by sampling two options (ensuring they're different from the correct answer).
    available_distractors = [d for d in distractor_bank if d.lower() != correct_translation.lower()]
    if len(available_distractors) < 2:
        available_distractors = distractor_bank
    distractors = random.sample(available_distractors, 2)
    options = distractors + [correct_translation]
    random.shuffle(options)

    return render_template_string("""
        <h1>Translate to French</h1>
        <p><strong>English:</strong> {{ english_sentence }}</p>
        <form method="POST">
            {% for opt in options %}
                <input type="radio" id="{{ opt }}" name="option" value="{{ opt }}" required>
                <label for="{{ opt }}">{{ opt }}</label><br>
            {% endfor %}
            <br>
            <input type="submit" value="Submit">
        </form>
        <p>Current Score: {{ score }}</p>
    """, english_sentence=english_sentence, options=options, score=session["score"])

def run_app():
    # Disable debug and reloader to avoid recursion errors in Colab.
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

# Start the Flask app in a background thread.
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


###Get the local tunnel password

In [17]:
!curl https://loca.lt/mytunnelpassword

34.87.44.15

###Run local tunnel

In [18]:
!/root/.npm-global/bin/lt --port 5000


your url is: https://cruel-rooms-cough.loca.lt


INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:15] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:30] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:41] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:55] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:57] "GET / HTTP/1.1" 200 -


^C
